In [1]:
%%capture

import warnings

warnings.filterwarnings("ignore")

from datetime import datetime

import altair as alt
import branca
import branca.colormap as cm
import calitp_data_analysis.magics
import geopandas as gpd
import google.auth
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from shared_utils import webmap_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

readable_dict = OmegaConf.load("new_readable.yml")
credentials, project = google.auth.default()

import _new_operator_report_utils as utils

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
def formatted(number):
    return "{:,}".format(number)

In [4]:
analysis_name = 'Yolo County Transportation District'

In [5]:
#  %%capture_parameters
#  analysis_name

In [6]:
# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [7]:
date = datetime.today().strftime("%Y_%m")

In [8]:
# Load datasets
schedule_rt_route_direction_summary = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/schedule_rt_route_direction_summary_{date}.parquet"
operator_hourly_summary = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/fct_operator_hourly_summary_{date}.parquet"

In [9]:
schedule_rt_route_direction_summary_df = pd.read_parquet(schedule_rt_route_direction_summary,filters=[[("Analysis Name", "==", analysis_name)]])

In [10]:
operator_hourly_summary_df = pd.read_parquet(operator_hourly_summary,filters=[[("Analysis Name", "==", analysis_name)]])

# {analysis_name}

## Service Hours 

In [11]:
utils.create_hourly_summary(operator_hourly_summary_df, "Weekday")

ConfigAttributeError: Missing key background_graph
    full_key: background_graph
    object_type=dict

In [ ]:
create_hourly_summary(sf_only, "Saturday")

In [ ]:
create_hourly_summary(sf_only, "Sunday")


## Detailed Route Overview

In [ ]:
utils.create_scheduled_minutes(schedule_rt_route_direction_summary_df)

In [ ]:
utils.create_scheduled_trips(schedule_rt_route_direction_summary_df)

In [ ]:
utils.create_frequency(schedule_rt_route_direction_summary_df)

In [ ]:
utils.create_text_graph(schedule_rt_route_direction_summary_df)